In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

import time
import math
import random
from tqdm import tqdm
import regex as re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

## Identify sequences that have always patterns and those who don't

In [2]:
always_patterns = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Data\always_patterns_8_1.csv")

In [3]:
always_pattern_regex = always_patterns["Pattern"].to_list()

In [4]:
for i in range(len(always_pattern_regex)):
    always_pattern_regex[i] = re.compile(always_pattern_regex[i], re.IGNORECASE)

In [6]:
train_df = pd.read_csv("Storage/Data/input.csv") 

train_df = train_df[['Unnamed: 0', 'patient_id', 'sequence','original', 'label']]
#train_df.columns = ['Unnamed: 0', 'patient_id', 'sequence','original', 'annotator_label']
train_df.head()

,Unnamed: 0,patient_id,sequence,original,label
0,0,Z15564314,s other free text-see phs viewer social histor...,s other free text-see phs viewer Social Histor...,1
1,1,Z10171706,------- fusion: no sleep disturbance: no socia...,------- fusion: No Sleep disturbance: No Socia...,1
2,2,Z8935348,ain spasm). 30 tablet 0 unknown (outside pharm...,ain/spasm). 30 tablet 0 Unknown (outside pharm...,1
3,3,Z12212893,------- on 112mcg dose. maria will check dose ...,------- on 112mcg dose. Maria will check dos...,2
4,4,Z9598376,------- (vibramycin) 100 mg capsule take 1 cap...,------- (VIBRAMYCIN) 100 MG capsule Take 1 ca...,1


In [10]:
a = []
c = []
counter = 0

for seq in tqdm(train_df["original"]):
    curr = []
    classes = []
    conversion = {2: "Yes", 1 : "Neither", 0: "No"}
    
    for p in (always_pattern_regex):
        m = list(set(re.findall(p, seq)))
        m = list(set(map(str.lower, m)))
        if (m != []):
            curr.append("".join(m))
            label = int(train_df.loc[counter]['label'])
            classes.append(conversion[label])
            classes = list(set(classes))
            
    a.append(curr)
    c.append(classes)
    counter += 1

train_df["always_pattern_match"] = a
train_df["always_pattern_classes"] = c

100%|████████████████████████████████████████████████████████████████████████████| 8656/8656 [00:07<00:00, 1152.80it/s]


In [12]:
train_df.head()

,Unnamed: 0,patient_id,sequence,original,label,always_pattern_match,always_pattern_classes
0,0,Z15564314,s other free text-see phs viewer social histor...,s other free text-see phs viewer Social Histor...,1,"[father, uncle, their first cousin, and his ma...",[Neither]
1,1,Z10171706,------- fusion: no sleep disturbance: no socia...,------- fusion: No Sleep disturbance: No Socia...,1,"[father with alzheimer's disease, father with ...",[Neither]
2,2,Z8935348,ain spasm). 30 tablet 0 unknown (outside pharm...,ain/spasm). 30 tablet 0 Unknown (outside pharm...,1,[family history: family history problem relati...,[Neither]
3,3,Z12212893,------- on 112mcg dose. maria will check dose ...,------- on 112mcg dose. Maria will check dos...,2,[cognitive deficits],[Yes]
4,4,Z9598376,------- (vibramycin) 100 mg capsule take 1 cap...,------- (VIBRAMYCIN) 100 MG capsule Take 1 ca...,1,[father history social history • marital statu...,[Neither]


In [14]:
# get sequences that were manually reviewed
manual_review = train_df[train_df["always_pattern_classes"].str.len() == 0]
manual_review.head()

,Unnamed: 0,patient_id,sequence,original,label,always_pattern_match,always_pattern_classes
75,75,Z10504958,ndings: general: no apparent distress. surgica...,ndings: General: No apparent distress. Surgica...,0,[],[]
77,77,Z6411816,"edness, no syncope, no dyspnea endocrine no po...","edness, No Syncope, No Dyspnea Endocrine No Po...",2,[],[]
87,87,Z6813204,5 reps calibrated hand gripper with silver spr...,5 reps Calibrated hand gripper with silver sp...,0,[],[]
95,95,Z10492528,nancial management services: none healthcare p...,nancial management Services: none Healthcare p...,0,[],[]
106,106,Z7656517,2 | 1 time: 1 | 0 time: 0) 3. “are there any r...,2 | 1 time: 1 | 0 time: 0) 3. “Are there any r...,0,[],[]


In [18]:
# get sequences that were annotated by always patterns 
always_patterns = train_df.drop(manual_review.index)
len(always_patterns)

8050

In [23]:
further_review = train_df["further_review"] = np.where((len(train_df["always_pattern_classes"]) > 1), 'N', 'Y')
further_review = train_df[train_df["further_review"] == "Y"]

In [24]:
further_review

,Unnamed: 0,patient_id,sequence,original,label,always_pattern_match,always_pattern_classes,further_review


In [25]:
manual_review.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Storage\Data\manual_review.csv", index = False)

In [26]:
always_patterns.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Storage\Data\always_patterns.csv", index = False)

## Clean 20K Sample

In [27]:
df = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\20K_sample_8_9.csv")

In [28]:
def clean_sequence(seq):
    #getting rid of special characters
    specials = '/' #etc
    seq_no_special_chars = seq.translate(str.maketrans(specials, ' '*len(specials)))
    
    #having only 1 space between words
    n = 1
    seq_no_spaces = (' '*n).join(seq_no_special_chars.split())
    
    return seq_no_spaces.lower()

In [29]:
for i in range(len(df)):
    df.loc[i, "regex_sent"] = clean_sequence(df.loc[i]["regex_sent"][7:len(df.loc[i]["regex_sent"]) - 5])

In [34]:
df = df[['PatientID', 'regex_sent']]

In [35]:
df.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Storage\Data\20K_sample.csv", index = False)

## Select new sample cohort (all sequences for all patients except those that were included in inital 8656)

In [9]:
entire = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\load_data\slat_8_16.csv", header = None)

In [46]:
len(entire), len(train_df)

(279224, 8656)

In [47]:
patient_id_included = train_df["patient_id"].unique().tolist()

In [48]:
entire["include"] = np.where(entire["PatientID"].isin(patient_id_included), 'Y', 'N')

In [49]:
new_sample = entire[entire["include"] == "N"]

In [51]:
len(new_sample["PatientID"].unique().tolist()), len(new_sample)

(13941, 186730)

In [53]:
new_sample.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Sampling\sample_8_12.csv", index = False)

## Ensuring All Sequences are Padded

In [3]:
train = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Storage\Data\train_full.csv")
test = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Storage\Data\test_full.csv")

In [4]:
len(train) + len(test)

8656

In [13]:
train

,patient_id,sequence,label
0,Z7805001,pleasant psychomotor: no abnormal movements st...,0
1,Z8435677,unknown (outside pharmacy) • oseltamivir (tami...,1
2,Z6738861,"------- 7), pulmonary emboli (2 27 2017), squa...",1
3,Z11869179,------- e reminded of their traumatic experien...,2
4,Z6545690,ately mental functions: arousal: intact orient...,0
...,...,...,...
8297,Z9337741,8 °c (98.2 °f) (12 01 18 1118) | heart rate: (...,0
8298,Z8146700,today past 24 hour overnight events: pt seen a...,0
8299,Z9328738,rmittently with tenderness to palpation in shi...,0
8300,Z8447004,"------- ith ecg of 30-aug-2018 12:00, sinus rh...",0


In [11]:
entire.columns = ["seq_number", "sequence", "note_id"]

In [20]:
test[test["sequence"].str.len() < 500]

,patient_id,sequence,label
166,Z8500820,----------------------------------------------...,2
234,Z8662811,----------------------------------------------...,1
263,Z8857259,----------------------------------------------...,0
301,Z8875000,----------------------------------------------...,0
